# Test Case LVV-T2578
This notebook tests the filter changer timing and position repeatability. In particular:

The requirement: LVV-14633 LTS-508-REQ-0015-V-01: Filter Changing_1
* Filter changer must be able to switch between filters remotely
* Positioning requirement is given in requirement LTS-508-4
* Filter change time: 30 s maximum
* Repeatability of motion: < +/- 0.1 mm (TBR)
* Filter installation positioning and reconfiguration: +/- 0.1 mm lateral, +/- 2.6 arcmin for rotation. (verified elsewhere)

In [ ]:
from lsst.sitcom import vandv

exec_info = vandv.ExecutionInfo()
print(exec_info)

## Setup

In [ ]:
import asyncio
import logging
import os
import yaml

import astropy.units as u
import numpy as np
import pandas as pd

from astropy.time import Time
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

from lsst.ts.observatory.control.maintel import ComCam
from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger
from lsst_efd_client import EfdClient
from lsst.ts import salobj

Setting up logger

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

Instantiate script for logging into EFD and start script task

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index = os.getuid() * 10 + np.random.randint(0, 9)

logger.info(f'The generated index is {index}')

In [ ]:
test_message = "LVV-T2578 ComCam OptoMechanical Filter Change Test"
script = salobj.Controller("Script", index=index)
await script.start_task

Make sure DDS Daemon is running and startup Domain

In [ ]:
domain = salobj.Domain()

EFD setup

In [ ]:
client = vandv.efd.create_efd_client()

ComCam initialization

In [ ]:
comcam = ComCam(domain=domain)
comcam.set_rem_loglevel(40)

In [ ]:
await comcam.start_task

In [ ]:
await comcam.enable()

Publish to the EFD that LVV-T2578 test is starting 

In [ ]:
script.log.info(f'START- {test_message} -- at {Time.now()}')

## Test

Get available instrument configurations and declare `filters`. 

In [ ]:
filters = await comcam.get_available_instrument_setup()
logger.info(f'Available filters are {filters}')

In [ ]:
# note that we are slightly out of spec for moving positions 0 to 2 and 2 to 0 by 5.3 seconds.  
# This might be due to the timing measurement.  Can we get get better measurments from the EFD?
# also need to grab the positions from the linear encoder to calculate the repeatability.

# Add linear encoder filter position for each move and timing from EFD events endSetFilter - setFilter. 

for i in range(0,10):   
    # Flush events
    comcam.rem.cccamera.evt_startSetFilter.flush()
    comcam.rem.cccamera.evt_endSetFilter.flush()
    
    # Get start time
    startdate = Time.now()
    
    # Change Filter
    await comcam.rem.cccamera.cmd_setFilter.set_start(name = filters[i % 3]) 

    # Record startSetFilter and endSetFilter events
    setFilter = await comcam.rem.cccamera.evt_startSetFilter.next(flush=False, timeout=10)
    #logger.info(setFilter)
    
    endSetFilter = await comcam.rem.cccamera.evt_endSetFilter.next(flush=False, timeout=40)
    #logger.info(endSetFilter)
    
    logger.info(f'Move to Slot {endSetFilter.filterSlot} \t Filter: {endSetFilter.filterName} \t '
                f'Filter Position Linear Encoder: {endSetFilter.filterPosition} [mm] \t ------ \t'
                f'Duration: {endSetFilter.private_sndStamp - setFilter.private_sndStamp:0.3f} [sec]')    
    
    # Get end time
    enddate = Time.now()
    print("Movement "+str(i)+" duration: "+str(24*60*60*(enddate-startdate))+" sec" )

Switching filters in the opposite direction

In [ ]:
for i in range(0,10):   
    # Flush events
    comcam.rem.cccamera.evt_startSetFilter.flush()
    comcam.rem.cccamera.evt_endSetFilter.flush()
    
    # Get start time
    startdate = Time.now()
    
    # Change Filter
    await comcam.rem.cccamera.cmd_setFilter.set_start(name = filters[(10 - i) % 3]) 

    # Record startSetFilter and endSetFilter events

    setFilter = await comcam.rem.cccamera.evt_startSetFilter.next(flush=False, timeout=10)
    # logger.info(setFilter)
    endSetFilter = await comcam.rem.cccamera.evt_endSetFilter.next(flush=False, timeout=40)
    # logger.info(endSetFilter)
    
    logger.info(f'Move to Slot {endSetFilter.filterSlot} \t Filter: {endSetFilter.filterName} \t '
                f'Filter Position Linear Encoder: {endSetFilter.filterPosition} [mm] \t ------ \t'
                f'Duration: {endSetFilter.private_sndStamp - setFilter.private_sndStamp:0.3f} [sec]')    
    
    # Get end time
    enddate = Time.now()
    print("Movement "+str(i)+" duration: "+str(24*60*60*(enddate-startdate)))

## Wrap-up

Announce the EFD that test is done 

In [ ]:
script.log.info(f'END- {test_message} -- at {Time.now()}')

Transition ComCam to STANDBY and close domain. 

In [ ]:
await comcam.standby()

In [ ]:
await domain.close()